# Import Libraries

In [4]:
import numpy as np
from scipy.signal import cont2discrete
from tabulate import tabulate
import math

![](img/InverterOut1.png)
![](img/InverterOut2.png)

The state-space model of the LCL filter for this project was modeled in LCL_Model.ipynb. In this project, we will obtain the A and B matrices based on the design parameters already determined.

$$
\dot{x} = Ax + Bu
$$

$$
y = Cx + Du
$$

$$\frac{d}{dt} \begin{bmatrix} i_{L_{1}}(t) \\ i_{L_{d}}(t) \\ i_{L_{2}}(t) \\ v_{C_{f}}(t) \\ v_{C_{d}}(t) \end{bmatrix} = 

\begin{bmatrix} -\frac{R_{1}}{L_{1}} & 0 & 0 & -\frac{1}{L_{1}} & 0 \\ 0 & 0 & 0 & \frac{1}{L_{d}} & -\frac{1}{L_{d}} \\ 0 & 0 & -\frac{R_{2}}{L_{2}} & \frac{1}{L_{2}} & 0 \\ \frac{1}{C_{f}} & -\frac{1}{C_{f}} & -\frac{1}{C_{f}} & -\frac{1}{C_{f} R_{d}} & \frac{1}{C_{f} R_{d}} \\ 0 & \frac{1}{C_{d}} & 0 & \frac{1}{C_{d} R_{d}} & -\frac{1}{C_{d} R_{d}} \end{bmatrix} \begin{bmatrix} i_{L_{1}}(t) \\ i_{L_{d}}(t) \\ i_{L_{2}}(t) \\ v_{C_{f}}(t) \\ v_{C_{d}}(t) \end{bmatrix}

+ \begin{bmatrix} \frac{1}{L_{1}} & 0 \\ 0 & 0 \\ 0 & -\frac{1}{L_{2}} \\ 0 & 0 \\ 0 & 0 \end{bmatrix} \begin{bmatrix} V_{1} \\ V_{2} \end{bmatrix} $$

# Parameters

In [5]:
# V1 = V_Link = 1500V

# LC Filter (Inverter Side)
R1 = 1e-6
L1 = 110e-6
Cf = 50e-6 * 3 # Delta Connection multiply *3

# Damping Filter
Cd = 25e-6
Rd = 2.5
Ld = 850e-6

# Grid Parameters
L2 = 34.5e3**2 / (8.33 * 4.42e6 * 2 * np.pi * 60)
R2 = 0.1

# V2
# 925V RMS na linha 
# 925V * sqrt(2) / sqrt(3) = 755V

# Calc Step
Ts = 500e-9

In [6]:
A = np.array([
    [-R1/L1, 0, 0, -1/L1, 0],
    [0, 0, 0, 1/Ld, -1/Ld],
    [0, 0, -R2/L2, 1/L2, 0],
    [1/Cf, -1/Cf, -1/Cf, -1/(Cf*Rd), 1/(Cf*Rd)],
    [0, 1/Cd, 0, 1/(Cd*Rd), -1/(Cd*Rd)],
])

B = np.array([
    [1/L1, 0],
    [0, 0],
    [0, -1/L2],
    [0, 0],
    [0, 0]
])

C = np.eye(5)
D = np.zeros((5, 2))

In [7]:
#print(A)
print("\n Matrix A =")
print(tabulate([[f"{num:.2e}" for num in linha] for linha in A], tablefmt="fancy_grid"))


 Matrix A =
╒════════════╤═══════╤══════════╤═════════╤════════╕
│   -0.00909 │     0 │     0    │ -9090   │      0 │
├────────────┼───────┼──────────┼─────────┼────────┤
│    0       │     0 │     0    │  1180   │  -1180 │
├────────────┼───────┼──────────┼─────────┼────────┤
│    0       │     0 │    -1.17 │    11.7 │      0 │
├────────────┼───────┼──────────┼─────────┼────────┤
│ 6670       │ -6670 │ -6670    │ -2670   │   2670 │
├────────────┼───────┼──────────┼─────────┼────────┤
│    0       │ 40000 │     0    │ 16000   │ -16000 │
╘════════════╧═══════╧══════════╧═════════╧════════╛


In [9]:
print(B)

[[9090.90909091    0.        ]
 [   0.            0.        ]
 [   0.          -11.66165528]
 [   0.            0.        ]
 [   0.            0.        ]]


# State Space Discretization

$$
\dot{x} = Ax + Bu
$$

$$
x_{k+1} = A_dx_k + B_du_k
$$

$$
y_k = C_d x_k + D_du_k
$$

In [40]:
Ad, Bd, _, _, _ = cont2discrete((A, B, C, D), Ts, method='euler')

In [41]:
print("\n Matrix Ad =")
print(tabulate([[f"{num:.5e}" for num in linha] for linha in Ad], tablefmt="fancy_grid"))

print("\n Matrix Bd =")
print(tabulate([[f"{num:.5e}" for num in linha] for linha in Bd], tablefmt="fancy_grid"))


 Matrix Ad =
╒════════════╤═════════════╤═════════════╤══════════════╤══════════════╕
│ 1          │  0          │  0          │ -0.00454545  │  0           │
├────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ 0          │  1          │  0          │  0.000588235 │ -0.000588235 │
├────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ 0          │  0          │  0.999999   │  5.83083e-06 │  0           │
├────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ 0.00333333 │ -0.00333333 │ -0.00333333 │  0.998667    │  0.00133333  │
├────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ 0          │  0.02       │  0          │  0.008       │  0.992       │
╘════════════╧═════════════╧═════════════╧══════════════╧══════════════╛

 Matrix Bd =
╒════════════╤══════════════╕
│ 0.00454545 │  0           │
├────────────┼──────────────┤
│ 0          │  0           │
├────────────┼──────────────┤
│ 0          │ -5.8

In [42]:
def float2Fixed(value, intBits, fracBits, signed=True):
    """
    Converte um número de ponto flutuante para ponto fixo com a quantidade de bits especificada.
    
    :param value: O número em ponto flutuante a ser convertido.
    :param intBits: Número de bits para a parte inteira.
    :param fracBits: Número de bits para a parte fracionária.
    :param signed: Se True, usa representação assinada; se False, usa representação não assinada.
    :return: O número em ponto fixo se estiver dentro do intervalo, ou uma mensagem de erro se extrapolar.
    """
    if intBits <= 0 or fracBits < 0:
        return "Número de bits deve ser positivo e parte fracionária não pode ser negativa."

    totalBits = intBits + fracBits
    factor = 2 ** fracBits

    if signed:
        maxIntValue = (2 ** (intBits - 1)) - 1
        minIntValue = -2 ** (intBits - 1)
    else:
        maxIntValue = (2 ** intBits) - 1
        minIntValue = 0

    # Verifica os limites do valor
    if value < minIntValue or value > maxIntValue + (1 - 1 / factor):
        return "Valor fora do intervalo permitido para a quantidade de bits especificada."

    # Converte o número para ponto fixo
    fixedValue = int(round(value * factor))

    # Verifica o intervalo do ponto fixo
    if signed:
        if fixedValue < (minIntValue * factor) or fixedValue > (maxIntValue * factor):
            return "Valor fora do intervalo permitido para a quantidade de bits especificada."
    else:
        if fixedValue < 0 or fixedValue > (maxIntValue * factor):
            return "Valor fora do intervalo permitido para a quantidade de bits especificada."

    return fixedValue

def convertMatrix(matrix, intBits, fracBits, signed=True):
    """
    Converte todos os elementos de uma matriz de ponto flutuante para ponto fixo.
    
    :param matrix: A matriz de ponto flutuante a ser convertida.
    :param intBits: Número de bits para a parte inteira.
    :param fracBits: Número de bits para a parte fracionária.
    :param signed: Se True, usa representação assinada; se False, usa representação não assinada.
    :return: A matriz convertida para ponto fixo.
    """
    # Itera sobre a matriz e aplica a conversão
    fixedMatrix = [[float2Fixed(element, intBits, fracBits, signed) for element in row] for row in matrix]
    
    return fixedMatrix

In [43]:
# Converter as matrizes para ponto fixo
intBits = 16
fracBits = 16
Ad_fp = convertMatrix(Ad, intBits, fracBits)
Bd_fp = convertMatrix(Bd, intBits, fracBits)

In [44]:
print("\n Matrix Ad_FxedPoint =")
print(tabulate([[f"{num:.5e}" for num in linha] for linha in Ad_fp], tablefmt="fancy_grid"))

print("\n Matrix Bd_FxedPoint =")
print(tabulate([[f"{num:.5e}" for num in linha] for linha in Bd_fp], tablefmt="fancy_grid"))


 Matrix Ad_FxedPoint =
╒═══════╤═══════╤═══════╤═══════╤═══════╕
│ 65536 │     0 │     0 │  -298 │     0 │
├───────┼───────┼───────┼───────┼───────┤
│     0 │ 65536 │     0 │    39 │   -39 │
├───────┼───────┼───────┼───────┼───────┤
│     0 │     0 │ 65536 │     0 │     0 │
├───────┼───────┼───────┼───────┼───────┤
│   218 │  -218 │  -218 │ 65449 │    87 │
├───────┼───────┼───────┼───────┼───────┤
│     0 │  1311 │     0 │   524 │ 65012 │
╘═══════╧═══════╧═══════╧═══════╧═══════╛

 Matrix Bd_FxedPoint =
╒═════╤═══╕
│ 298 │ 0 │
├─────┼───┤
│   0 │ 0 │
├─────┼───┤
│   0 │ 0 │
├─────┼───┤
│   0 │ 0 │
├─────┼───┤
│   0 │ 0 │
╘═════╧═══╛


In [45]:
def dec2Hex(numero, bits):
    """
    Converte um número decimal (positivo ou negativo) para hexadecimal com uma quantidade específica de bits, usando complemento de dois.

    Args:
    numero (int): O número decimal a ser convertido.
    bits (int): A quantidade de bits a ser utilizada.

    Returns:
    str: O número hexadecimal representado como uma string com a quantidade de bits especificada.
    """
    # Calcula o número máximo e mínimo representável com a quantidade de bits
    max_valor = (1 << bits) - 1
    min_valor = - (1 << (bits - 1))
    
    # Garante que o número esteja dentro do intervalo permitido
    if not (min_valor <= numero <= max_valor):
        raise ValueError(f"O número deve estar entre {min_valor} e {max_valor} para {bits} bits.")
    
    # Se o número é negativo, convertemos para complemento de dois
    if numero < 0:
        numero = (1 << bits) + numero
    
    # Converte o número para hexadecimal e formata com a quantidade de bits
    hexadecimal = format(numero, 'X').zfill(bits // 4)
    
    return hexadecimal


In [46]:
totalBits = intBits + fracBits
Ad_fpHex = np.array([[dec2Hex(x, totalBits) for x in linha] for linha in Ad_fp])
Bd_fpHex = np.array([[dec2Hex(x, totalBits) for x in linha] for linha in Bd_fp])

In [48]:
Ad_fpHex

array([['00010000', '00000000', '00000000', 'FFFFFED6', '00000000'],
       ['00000000', '00010000', '00000000', '00000027', 'FFFFFFD9'],
       ['00000000', '00000000', '00010000', '00000000', '00000000'],
       ['000000DA', 'FFFFFF26', 'FFFFFF26', '0000FFA9', '00000057'],
       ['00000000', '0000051F', '00000000', '0000020C', '0000FDF4']],
      dtype='<U8')

In [49]:
Bd_fpHex

array([['0000012A', '00000000'],
       ['00000000', '00000000'],
       ['00000000', '00000000'],
       ['00000000', '00000000'],
       ['00000000', '00000000']], dtype='<U8')

In [52]:
dec2Hex(float2Fixed(-1500/2, intBits, fracBits), totalBits)

'FD120000'